In [27]:
import numpy as np

In [3]:
import pandas as pd
df=pd.read_csv("merged_CSVs_fixed_duplicate_headers.csv")
df

,flow_id,timestamp,src_ip,src_port,dst_ip,dst_port,protocol,duration,packets_count,fwd_packets_count,...,max_fwd_payload_bytes_delta_len,mean_fwd_payload_bytes_delta_len,mode_fwd_payload_bytes_delta_len,variance_fwd_payload_bytes_delta_len,std_fwd_payload_bytes_delta_len,median_fwd_payload_bytes_delta_len,skewness_fwd_payload_bytes_delta_len,cov_fwd_payload_bytes_delta_len,label,activity
0,35.203.211.133_54573_10.0.4.57_25094_TCP_2023-...,2023-12-14 09:01:03.508091,35.203.211.133,54573,10.0.4.57,25094,TCP,0.000063,3,2,...,0.0,0.0,0.0,0.0000,0.0000,0.0,NaN,NaN,Benign,Benign
1,10.0.4.57_25094_35.203.211.133_54573_TCP_2023-...,2023-12-14 09:01:03.508156,10.0.4.57,25094,35.203.211.133,54573,TCP,0.000000,1,0,...,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,Benign,Benign
2,35.203.211.133_54573_10.0.4.57_25094_TCP_2023-...,2023-12-14 09:01:03.508431,35.203.211.133,54573,10.0.4.57,25094,TCP,0.000028,3,1,...,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,Benign,Benign
3,162.142.125.181_9147_10.0.4.57_18060_TCP_2023-...,2023-12-14 09:01:06.696817,162.142.125.181,9147,10.0.4.57,18060,TCP,0.000055,3,2,...,0.0,0.0,0.0,0.0000,0.0000,0.0,NaN,NaN,Benign,Benign
4,10.0.4.57_18060_162.142.125.181_9147_TCP_2023-...,2023-12-14 09:01:06.696874,10.0.4.57,18060,162.142.125.181,9147,TCP,0.000000,1,0,...,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,Benign,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700769,60.248.107.64_46689_10.0.3.52_445_TCP_2023-12-...,2023-12-19 17:00:01.184440,60.248.107.64,46689,10.0.3.52,445,TCP,1.104332,14,7,...,296.0,0.0,-454.0,50455.6667,224.6234,42.5,-0.9866,inf,Benign,Benign-Systemic
700770,60.248.107.64_46724_10.0.3.52_445_TCP_2023-12-...,2023-12-19 17:00:02.241538,60.248.107.64,46724,10.0.3.52,445,TCP,1.233900,13,7,...,296.0,0.0,-454.0,50455.6667,224.6234,42.5,-0.9866,inf,Benign,Benign-Systemic
700771,77.90.185.48_48509_10.0.9.208_30775_TCP_2023-1...,2023-12-19 17:00:02.362816,77.90.185.48,48509,10.0.9.208,30775,TCP,0.000000,1,1,...,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0,Suspicious,Suspicious
700772,60.248.107.64_46751_10.0.3.52_445_TCP_2023-12-...,2023-12-19 17:00:03.278275,60.248.107.64,46751,10.0.3.52,445,TCP,0.205192,5,3,...,73.0,36.5,0.0,1332.2500,36.5000,36.5,0.0000,1.0,Benign,Benign-Systemic


In [5]:
df.shape

(700774, 324)

In [7]:
df.columns = df.columns.str.strip()
#removes leading and trailing spaces from column names

In [9]:
with pd.option_context('display.max_rows', 50):
    print(df.isna().sum().sort_values(ascending=False).head(50))

bwd_packets_IAT_skewness                540398
payload_bytes_skewness                  540271
payload_bytes_cov                       540111
fwd_packets_IAT_skewness                496055
fwd_skewness_header_bytes               477282
fwd_payload_bytes_skewness              476027
fwd_payload_bytes_cov                   475285
skewness_header_bytes                   456723
packets_IAT_skewness                    449922
bwd_packets_IAT_cov                     345715
skewness_payload_bytes_delta_len        210316
cov_payload_bytes_delta_len             210016
bwd_payload_bytes_skewness              196913
bwd_payload_bytes_cov                   196675
skewness_header_bytes_delta_len         180376
bwd_skewness_header_bytes               175973
skewness_packets_delta_len              161362
cov_header_bytes_delta_len              126468
skewness_packets_delta_time             119741
skewness_fwd_header_bytes_delta_len      81862
cov_fwd_header_bytes_delta_len           81405
skewness_fwd_

In [11]:
df.isna().any(axis=1).sum()
#no. of rows with atleast one NaN value

544974

In [15]:
#filling the NaN because majority elements have NaN
df['timestamp'] = df['timestamp'].fillna(value='')
df.fillna(value=0.0, inplace=True)
print(df.isna().any(axis=1).sum(), "rows with at least one NaN to remove")

0 rows with at least one NaN to remove


In [17]:
df.duplicated().sum()
#duplicate rows

322

In [19]:
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)
#dropping the duplicate rows

In [21]:
df = df.drop(columns=['flow_id', 'timestamp', 'src_ip', 'dst_ip'])
#contaminating features are dropped

In [23]:
df['delta_start'] = df['delta_start'].replace(to_replace='not a complete handshake', value=-150.0)
#the delta_start has 'not a complete handshake' as one value, so replacing with unique value -150

In [29]:
df['delta_start'] = df['delta_start'].astype(np.float32)

In [31]:
df.handshake_duration.value_counts()

handshake_duration
not a complete handshake    542317
0.1111                        4178
0.1112                        3455
0.1113                        3043
0.1110                        2483
                             ...  
0.3199                           1
1.3306                           1
1.3441                           1
0.4110                           1
1.2069                           1
Name: count, Length: 4306, dtype: int64

In [33]:
df['handshake_duration'] = df['handshake_duration'].replace(to_replace='not a complete handshake', value=-1.0)
df['handshake_duration'] = df['handshake_duration'].astype(np.float32)
#same kind of issue

In [39]:
df.shape

(700452, 320)

In [41]:
df = df.drop(columns=['protocol'])
#Also irrelevant feature

In [43]:
df.to_csv('cleanedData.csv')